In [1]:
import os
from pathlib import Path

os.chdir(Path.cwd().parent)
print("Working directory is now:", Path.cwd())


Working directory is now: c:\Users\abdulhamed\Desktop\depi\DEPI-GP


In [2]:
import json
from pathlib import Path


In [3]:
raw_dir = Path('raw_outputs')
data_dir = Path('data')

paths = {
    'topics': raw_dir / 'egy_agri_topics_final_sorted.jsonl',
    'videos': raw_dir / 'videos_data-annotated.jsonl',
    'wiki': raw_dir / 'wiki_QA_responses.jsonl',
}

output_path = data_dir / 'agri_sft.jsonl'

def load_jsonl(path):
    with path.open('r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                yield json.loads(line)


In [4]:
prefixes = ["اشرح ", "ما هو ", "يعني اي", "كلمني عن ", "ما المقصود ب "]

topic_examples = []
for i, rec in enumerate(load_jsonl(paths['topics'])):
    topic = (rec.get('topic') or '').strip()
    subtopic = (rec.get('subtopic') or '').strip()
    if not topic or not rec.get('text'):
        continue
    prompt = prefixes[i % len(prefixes)] + topic
    if subtopic:
        prompt += " - " + subtopic
    topic_examples.append({
        'source': 'egy_topics',
        'messages': [
            {'role': 'user', 'content': prompt},
            {'role': 'assistant', 'content': rec['text']},
        ],
    })

print('Topics:', len(topic_examples))


Topics: 447


In [6]:
video_examples = []
for rec in load_jsonl(paths['videos']):
    conversation = (rec.get('annotation') or {}).get('conversation') or []
    if not isinstance(conversation, list) or not conversation:
        continue
    messages = []
    for turn in conversation:
        role = turn.get('role')
        content = turn.get('content')
        if role in {'user', 'assistant'} and content:
            messages.append({'role': role, 'content': content})
    if len(messages) < 2:
        continue
    video_examples.append({
        'source': 'youtube',
        'messages': messages,
    })
print('Videos:', len(video_examples))

Videos: 49


In [7]:

wiki_examples = []
for rec in load_jsonl(paths['wiki']):
    question = rec.get('question')
    response = rec.get('response')
    if not question or not response:
        continue
    wiki_examples.append({
        'source': 'wikipedia',
        'messages': [
            {'role': 'user', 'content': question},
            {'role': 'assistant', 'content': response},
        ],
    })

print('Wikipedia:', len(wiki_examples))


Wikipedia: 513


In [8]:
examples = topic_examples + video_examples + wiki_examples
for idx, ex in enumerate(examples, start=1):
    ex['id'] = str(idx)

output_path.parent.mkdir(parents=True, exist_ok=True)
with output_path.open('w', encoding='utf-8') as f:
    for ex in examples:
        f.write(json.dumps(ex, ensure_ascii=False) + '\n')

print('Total:', len(examples))
print('Wrote:', output_path)


Total: 1009
Wrote: data\agri_sft.jsonl


In [11]:
from collections import Counter

counts = Counter(ex['source'] for ex in examples)
print('Counts:', counts)


for src in counts.keys():
    ex = next((e for e in examples if e.get("source") == src), None)
    if ex is None:
        print(f"\n--- No example found for source: {src} ---")
        continue

    print(f"\n--- Example from source: {src} ---")
    print(json.dumps(ex, ensure_ascii=False)[:500] + "...")


Counts: Counter({'wikipedia': 513, 'egy_topics': 447, 'youtube': 49})

--- Example from source: egy_topics ---
{"source": "egy_topics", "messages": [{"role": "user", "content": "اشرح الفلاح المصري - الفلاح المصري"}, {"role": "assistant", "content": "## الفلاح المصري: صورة من الواقع الزراعي في قلب مصر\n\n### 1. مقدمة عن الفلاح المصري ودوره في المجتمع\n\nالفلاح المصري هو شريحة أساسية من نسيج الشعب، وهو العمود الفقري للاقتصاد الزراعي الذي يُعدّ أحد أهم مصادر الدخل القومي. رغم أن مصر تُصنّف كدولة صناعية في كثير من الإحصاءات، فإن أكثر من نصف السكان ما زالوا يعتمدون على الزراعة في سبل عيشهم، سواءً كان ذلك عبر ...

--- Example from source: youtube ---
{"source": "youtube", "messages": [{"role": "user", "content": "هناك من يقول إن زراعة أشجار الليمون في البراميل أو الأوعية الكبيرة تكون زراعة فاشلة، ولا تنتج غزارة مثل الأشجار المزروعة في الأرض، وأن كل التعب يكون على الفاضي. هل هذا صحيح؟ وكيف يمكن لبرميل أن ينتج محصولاً وفيراً من الليمون؟"}, {"role": "assistant", "content": "الذي يقول ذلك إما لم